In [1]:
%pip install nltk


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import nltk
from nltk.corpus import movie_reviews
nltk.download('movie_reviews')

[nltk_data] Downloading package movie_reviews to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


True

In [3]:
# make all the bigrams for this entire corpus
# there's a method in nltk that doe sthis for us!
review_bigrams = nltk.bigrams(movie_reviews.words())

In [16]:
# now we want to build a full bigram model for all context tokens
from collections import Counter

# build the bigrams again
review_bigrams = list(nltk.bigrams(movie_reviews.words()))[:1000]

# use a dictionary to keep track of all the context tokens
bigram_lm = {}
target_tokens = {}

# figure out all the word types first
vocabulary = set([word.lower() for word in movie_reviews.words()])

# build the bigram model
for bigram in review_bigrams:
    context_token = bigram[0].lower()
    target_token = bigram[1].lower()
    if context_token in target_tokens:
        target_tokens[context_token].append(target_token)
    else:
        target_tokens[context_token] = [target_token]

for context_token in target_tokens.keys():
    bigram_lm[context_token] = Counter(target_tokens)

# What did we build? Let's take a look at the bigram model
print(bigram_lm['people'].most_common(10))
print(bigram_lm['what'].most_common(10))
print(bigram_lm['people']['are'])

[('with', ['your', 'the', 'movies', 'little', 'the']), ('enter', ['your']), ('also', ['wrapped']), ('mess', ['with']), ('problems', ['with']), ('line', ['with']), ('video', ['with']), ('hot', ['with']), ('blair', ['witch']), ('know', ['who', 'the', 'why'])]
[('with', ['your', 'the', 'movies', 'little', 'the']), ('enter', ['your']), ('also', ['wrapped']), ('mess', ['with']), ('problems', ['with']), ('line', ['with']), ('video', ['with']), ('hot', ['with']), ('blair', ['witch']), ('know', ['who', 'the', 'why'])]
['good', 'the', 'dreams', 'characters', 'others', 'strange', 'disappearances', 'a', 'tons', 'given', 'people', '.', 'pretty']
